In [1]:
import nltk
import re
import sklearn.manifold

In [2]:
from collections import defaultdict

In [3]:
f = open("pba.txt", "r")
txt = f.read()

In [4]:
# Tokenizing
tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
raw = tokenizer.tokenize(txt)

In [5]:
print(raw)

['-\n"Lo que sostiene a la pareja es el amor"\nClara Crespo (50) y Rodolfo Martínez (54) no se imaginan uno sin el otro.', '"Prefiero ni pensarlo", dice Clara.', 'Hace 26 años que están casados, y tienen cuatro hijas mujeres.', 'Se conocieron en el Ateneo Juventus, el movimiento juvenil de Capuchinos.', 'Hoy aseguran no estar sorprendidos del tiempo que llevan juntos sino de haber logrado entenderse tan bien.', '&#226;&#8364;&#8220;¿Qué les gusta y disgusta del otro?', '¿Qué quisieran cambiarle?', '&#226;&#8364;&#8220;Rodolfo: Me gusta que sea cariñosa, alegre y esté siempre pensando en mí, y que es una gran madre.', 'Me disgustaba que cuando se enojaba no quería hablar, pero ya no lo hace más.', 'A veces es indecisa pero ya me acostumbré.', 'No quiero cambiarle nada, que sea como es.', '&#226;&#8364;&#8220;Clara : Me gusta que es una persona emprendedora, alegre, optimista y servicial.', 'Me gustaría que a veces fuera más sutil para decir las cosas.', 'Pienso que las personas vamos ca

In [6]:
# Cleaning the corpus (. , ! ? ...)
words = []
for s in raw:
    #s = re.sub(r"\d+", "NUM", s)
    corpus = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]", " ", s)
    corpus_sin_espacios = re.sub(' +',' ',corpus)
    #corpus = corpus.lower()
    #corpus = corpus.split()                              ----------> ojo 
    #corpus = re.findall(r'\S+', corpus,flags = re.UNICODE)
    words.append(corpus_sin_espacios)
    
#print(words)

In [7]:
print(words[:5])

[' Lo que sostiene a la pareja es el amor Clara Crespo y Rodolfo Martínez no se imaginan uno sin el otro ', ' Prefiero ni pensarlo dice Clara ', 'Hace años que están casados y tienen cuatro hijas mujeres ', 'Se conocieron en el Ateneo Juventus el movimiento juvenil de Capuchinos ', 'Hoy aseguran no estar sorprendidos del tiempo que llevan juntos sino de haber logrado entenderse tan bien ']


In [8]:
sent_list = []
for sentence in words:
    if sentence != ' ':
        sent_list.append([sentence])

In [9]:
sent_list[:5]

[[' Lo que sostiene a la pareja es el amor Clara Crespo y Rodolfo Martínez no se imaginan uno sin el otro '],
 [' Prefiero ni pensarlo dice Clara '],
 ['Hace años que están casados y tienen cuatro hijas mujeres '],
 ['Se conocieron en el Ateneo Juventus el movimiento juvenil de Capuchinos '],
 ['Hoy aseguran no estar sorprendidos del tiempo que llevan juntos sino de haber logrado entenderse tan bien ']]

In [10]:
final = []
for sent in sent_list:
    #print(sent[0])
    aux = sent[0].split()
    #print(aux)
    if  len(aux[0]) == 1:
        #print(aux[0])
        if aux[0] not in {'A', 'Y', 'O'}:
            #print(aux[0])
            del aux[0]
    final.append(aux)


In [11]:
print(final[:5])

[['Lo', 'que', 'sostiene', 'a', 'la', 'pareja', 'es', 'el', 'amor', 'Clara', 'Crespo', 'y', 'Rodolfo', 'Martínez', 'no', 'se', 'imaginan', 'uno', 'sin', 'el', 'otro'], ['Prefiero', 'ni', 'pensarlo', 'dice', 'Clara'], ['Hace', 'años', 'que', 'están', 'casados', 'y', 'tienen', 'cuatro', 'hijas', 'mujeres'], ['Se', 'conocieron', 'en', 'el', 'Ateneo', 'Juventus', 'el', 'movimiento', 'juvenil', 'de', 'Capuchinos'], ['Hoy', 'aseguran', 'no', 'estar', 'sorprendidos', 'del', 'tiempo', 'que', 'llevan', 'juntos', 'sino', 'de', 'haber', 'logrado', 'entenderse', 'tan', 'bien']]


In [12]:
from nltk.tag.stanford import StanfordPOSTagger

path_to_model = "/home/lgg/Documentos/facu/mdt/stanford-postagger-full-2017-06-09/models/spanish-distsim.tagger"
path_to_jar = "/home/lgg/Documentos/facu/mdt/stanford-postagger-full-2017-06-09/stanford-postagger-3.8.0.jar"

tagger = StanfordPOSTagger(path_to_model, path_to_jar)

In [13]:
a = []
for sent in final:
    a = a + ([tagger.tag(sent)])

In [14]:
print(a[0])

[('Lo', 'da0000'), ('que', 'pr000000'), ('sostiene', 'vmip000'), ('a', 'sp000'), ('la', 'da0000'), ('pareja', 'nc0s000'), ('es', 'vsip000'), ('el', 'da0000'), ('amor', 'nc0s000'), ('Clara', 'np00000'), ('Crespo', 'np00000'), ('y', 'cc'), ('Rodolfo', 'np00000'), ('Martínez', 'np00000'), ('no', 'rn'), ('se', 'p0000000'), ('imaginan', 'vmip000'), ('uno', 'pi000000'), ('sin', 'sp000'), ('el', 'da0000'), ('otro', 'pi000000')]


In [15]:
#rebie lista de tuplas
#Dos modos -> "w" = palabra anterior , "t" = tag anterior
def tw_before(lists, i, mode):
    if i != 0:
        if mode == "w":
            return lists[i-1][0] #palabra
        else:
            return lists[i-1][1] #tag
    else:
        return "[start]"

In [16]:
#recibe lista de tuplas
#Dos modos -> "w" = palabra siguiente , "t" = tag siguiente
def tw_after(lists, i, mode):
    if i != len(lists)-1:
        if mode == "w":
            return lists[i+1][0] #palabra
        else:
            return lists[i+1][1] #tag
    else:
        return "[end]"

In [17]:
#lematizado
lemma_dict = defaultdict(str)
with open('lemmatization-es.txt') as f:
    for line in f:
        splited_line = line.split('\t')
        #print(splited_line)
        lemma_dict[splited_line[1].strip()] = splited_line[0]

In [18]:
def lemmatize(word):
   return lemma_dict.get(word, word) # + u'*'

In [19]:
features = []
check_list = []

In [20]:
for sent in a:
    for i, (fst,snd) in enumerate(sent):
        new_stuff_dict = {'lemma':lemmatize(fst.lower()),'POS':snd, 'lowcase':fst.islower(), "wbefore":tw_before(sent, i, "w"), "tbefore":tw_before(sent, i, "t"), "wafter":tw_after(sent, i, "w"), "tafter":tw_after(sent,i,"t")} # la palabra en si no es un feature
        features.append(new_stuff_dict)
        check_list.append(fst)

In [21]:
features[:3]

[{'POS': 'da0000',
  'lemma': 'el',
  'lowcase': False,
  'tafter': 'pr000000',
  'tbefore': '[start]',
  'wafter': 'que',
  'wbefore': '[start]'},
 {'POS': 'pr000000',
  'lemma': 'que',
  'lowcase': True,
  'tafter': 'vmip000',
  'tbefore': 'da0000',
  'wafter': 'sostiene',
  'wbefore': 'Lo'},
 {'POS': 'vmip000',
  'lemma': 'sostener',
  'lowcase': True,
  'tafter': 'sp000',
  'tbefore': 'pr000000',
  'wafter': 'a',
  'wbefore': 'que'}]

In [22]:
#Vectorizado
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
X = v.fit_transform(features)

In [23]:
Y = v.inverse_transform(X)

In [24]:
Y[1]

{'POS=pr000000': 1.0,
 'lemma=que': 1.0,
 'lowcase': 1.0,
 'tafter=vmip000': 1.0,
 'tbefore=da0000': 1.0,
 'wafter=sostiene': 1.0,
 'wbefore=Lo': 1.0}

In [25]:
X[1]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [26]:
from sklearn.decomposition import TruncatedSVD

In [27]:
#Reducción de dimensionalidad
svd = TruncatedSVD(n_components=300, n_iter=7, random_state=42)
svd.fit_transform(X)

array([[  2.09940083e-01,  -3.38443935e-01,   1.08521530e+00, ...,
          4.38897200e-03,  -5.16779489e-03,   4.61525147e-04],
       [  1.10759971e+00,   2.46312529e-01,  -3.04492855e-01, ...,
         -2.28610849e-02,  -4.05514530e-02,  -1.00689347e-01],
       [  1.16708360e+00,   3.63994144e-01,  -2.59453003e-01, ...,
          6.11273334e-02,  -1.01858439e-01,  -3.34406722e-02],
       ..., 
       [  1.01558927e+00,  -2.06248423e-02,  -8.21721261e-02, ...,
         -1.58584956e-02,   3.15783181e-02,   7.94649420e-03],
       [  1.00984036e+00,  -8.33571674e-02,  -1.37053750e-01, ...,
          3.69209782e-03,   9.54952140e-04,   1.99159083e-02],
       [  1.12740250e+00,   2.86740633e-01,  -6.91632909e-02, ...,
          4.90407406e-03,  -4.31826869e-03,   2.13795576e-02]])

In [28]:
X[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [29]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=40)
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=40, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [30]:
labels = km.predict(X)

In [31]:
print(labels)

[28 17 19 ..., 23 32 30]


In [32]:
#Clusters obtenidos
words = defaultdict(set)
for i, label in enumerate(labels):
    words[label].add(check_list[i])
print(words)

defaultdict(<class 'set'>, {28: {'Creo', 'Creen', 'Hoy', 'Nunca', 'Seguramente', 'Hay', 'Lo', 'Después', 'Siempre', 'Menos', 'Pienso', 'Así', 'También', 'Nada'}, 17: {'que', 'cuando'}, 19: {'sea', 'molestan', 'vivimos', 'les', 'fue', 'sostiene', 'en', 'levantaban', 'ver', 'más', 'ya', 'abrimos', 'anticipa', 'están', 'se', 'calificaron', 'vino', 'mantiene', 'llevan'}, 14: {'a'}, 4: {'la', 'los', 'las'}, 16: {'mujer', 'forma', 'medida', 'tormenta', 'día', 'pelo', 'pareja', 'otro', 'triunfo', 'vida', 'toma', 'llama', 'edad', 'pasión', 'acuerdo', 'entrada', 'grupo', 'acta', 'primer', 'duración', 'reclamo', 'secreto', 'sábado', 'momento', 'charla', 'amor', 'ley', 'agua', 'café', 'otros', 'luna', 'unidad', 'anteproyecto', 'miércoles', 'tiempo', 'movimiento'}, 32: {'somos', 'fue', 'fueron', 'Somos', 'son', 'ser', 'es', 'sido'}, 9: {'lo', 'el'}, 10: {'Ipem', 'Jerónimo', 'Fábrica', 'Nicolás', 'Deán', 'Secundarios', 'Coordinadora', 'Clara', 'Ateneo', 'Villa', 'Ministerio', 'Pedro', 'María', 'Ale

In [74]:
print(words[23])

{'Carbó', 'Belgrano', 'Gordita', 'Rodríguez', 'Crespo', 'Cabrera', 'Educación', 'Luis', 'Cornú', 'Funes', 'Martínez', 'Copérnico', 'Interestudiantil', 'Virgen', 'Unidos', 'Mary', 'Juventus', 'Dios', 'Ipem', 'Ministerio', 'Pedro', 'Barrionuevo'}


In [67]:
print(words[13])

{'fue', 'se', 'molestan', 'vino', 'sostiene', 'ya', 'llevan', 'calificaron', 'levantaban', 'en', 'han', 'están', 'mantiene', 'les', 'anticipa', 'ver', 'sea', 'abrimos', 'vivimos', 'más'}


In [68]:
print(words[32])

{'se', 'Los', 'le', 'Me', 'lo', 'les', 'te', 'nos', 'los', 'la', 'me'}


In [106]:
print(words[15])

{'o', 'pero', 'y'}


In [141]:
print(words[6])

{'fue', 'fueron', 'es', 'Somos', 'son', 'sido', 'somos', 'ser'}


In [29]:
print(words[25])

{'radio', 'felicidad', 'reclamo', 'entrada', 'pasión', 'misma', 'vida', 'anteproyecto', 'normativa', 'medida', 'unidad', 'toma', 'tiempo', 'secreto', 'luna', 'muerte', 'duración', 'forma', 'mujer', 'edad', 'tormenta', 'ley'}
